The objective of this code is to extract aspect,aspect category and their sentiment score using pretrained "FINBERT" model which is just BERT model
trained on Financal Dataset

FOR DATASET PLEASE GO THE KAGGLE LINK PROVIDED BELOW. INSTEAD OF THAT ITS BETTER TO RUN IT IN KAGGLE

NOTE- It is ideal to run this on KAGGLE as it requires GPU . THE LINK TO THE CODE IS   https://www.kaggle.com/code/adi253/aspect-extraction.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fiqadataset/tatasentiment.csv
/kaggle/input/fiqadataset/apollosentiment.csv
/kaggle/input/fiqadataset/reliancesentiment.csv


In [2]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification  # import hugguging face libraries for running the model
import torch.nn.functional as F
from transformers import pipeline # to create a pipeline for sentiment score 

<font size="5">Financial aspect words and their categories.</font>

<font size="3">We have created this by taking most used finance words from the news dataset and also by generating using LLM models.
We have classified these aspect to  12 general financial categories so that when we create sentiment score for aspects we use these categories for further understanding.</font>

In [3]:
## Possible aspect words and their categories 

implicit_aspect_categories = {
    'stock_market': ['surge', 'plummet', 'soar', 'dropped', 'dipped', 'rallied', 'slumped', 
                     'volatile', 'bullish', 'bearish', 'all-time high', 'record low', 
                     'gains', 'losses', 'market sentiment', 'market reaction', 
                     'trading volume', 'price action', 'stake', 'ownership', 'shareholding', 
                     'equity', 'market capitalization', 'trending'],

    'financial_performance': ['beat expectations', 'missed projections', 'cost-cutting', 
                              'revenue growth', 'margin pressure', 'profit warning', 
                              'earnings surprise', 'outperformed', 'underperformed', 
                              'strong performance', 'weak performance', 'increased demand', 
                              'declining demand', 'operating loss', 'profit growth',
                              'financial results', 'net income', 'earnings per share', 
                              'profit margin', 'return on equity', 'gross profit',"expenses","quarter"],

    'investor_sentiment': ['positive outlook', 'negative outlook', 'investor confidence', 
                           'market confidence', 'bearish sentiment', 'bullish sentiment', 
                           'analyst upgrade', 'analyst downgrade', 'investor fears', 
                           'market concerns', 'sentiment shift', 'enthusiasm', 
                           'cautious', 'bull run', 'bear market','shareholders',"shareholders"],

    'market_trends': ['sector rotation', 'market correction', 'upward momentum', 
                      'downward pressure', 'overbought', 'oversold', 'price momentum', 
                      'breakout', 'pullback', 'technical indicators', 'trend reversal', 
                      'market dynamics', 'market breadth', 'momentum', 'futures trend'],

    'economic_indicators': ['inflation concerns', 'interest rate hike', 'monetary policy shift', 
                            'economic slowdown', 'recession fears', 'growth prospects', 
                            'GDP growth', 'inflationary pressure', 'stimulus impact', 
                            'fiscal policy change', 'employment data', 'consumer spending', 
                            'economic forecasts', 'interest rates', 'consumer price index', 
                            'purchasing power'],

    'corporate_news': ['executive change', 'leadership transition', 'strategic shift', 
                       'merger talks', 'acquisition rumors', 'partnership announcement', 
                       'divestiture', 'restructuring', 'new product launch', 
                       'regulatory approval', 'litigation risk', 'legal challenges', 
                       'board decision', 'management change', 'company strategy', 
                       'business restructuring'],

    'financial_crisis': ['liquidity crisis', 'credit freeze', 'market panic', 
                         'volatility spike', 'crisis unfolding', 'financial uncertainty', 
                         'market instability', 'sell-off', 'bank failure', 
                         'economic meltdown', 'financial contagion', 'recession', 'defaults'],

    'commodities': ['oil price surge', 'commodity price hike', 'commodity price drop', 
                    'supply chain disruption', 'global demand', 'commodity inflation', 
                    'raw material cost', 'gold prices', 'silver prices', 
                    'agricultural commodities', 'metals market'],

    'debt_credit': ['debt levels', 'loan approvals', 'bond yields', 'credit risk', 
                    'default rates', 'interest payments', 'refinancing', 
                    'credit downgrade', 'debt restructuring'],

    'regulatory_legal': ['regulatory compliance', 'legal challenges', 'government regulations', 
                         'lawsuit settlement', 'investigation results', 'regulatory scrutiny'],

    'corporate_actions': ['dividend announcement', 'stock split', 'rights issue', 
                          'buyback', 'initial public offering', 'public offering', 
                          'investment strategy'],
    "companies":['Apollo Hospitals',"Reliance","Tata","Tata Motors","Reliance Jio"]

}


In [4]:
# additional sets we are adding 

new_aspects = {
    'stock_market': ['sensex', 'index', 'points', 'stocks', 'shares', 'nifty'],
    'financial_performance': [ 'turnover', 'earnings report', 'dividend', 'margin', 'profit margin'],
    'corporate_news': ['reliance jio', 'reliance retail', 'ipo', 'fpo', 'capital raise', 'new initiative'],
    'market_trends': [ 'growth stocks', 'blue-chip', 'small-cap', 'large-cap', 'mid-cap'],
    'investor_sentiment': ['mukesh ambani', 'market speculation', 'rumors', 'investor reactions'],
    'economic_indicators': ['gdp numbers', 'inflation rate', 'monetary easing', 'economic recovery','net profit']
 
}


for category, new_terms in new_aspects.items():
    implicit_aspect_categories[category].extend(new_terms)

<font size="5"> LOADING FINBERT MODEL </font>

<font size="3"> We are using FINBERT Model which is a BERT model trained on finance dataset for giving sentiment score as well as sentiment category for the text based on the aspect word.
We are creating a pipeline for the model for inference.</font>

In [5]:
import spacy  # Import the spaCy library for natural language processing
from transformers import pipeline  # Import the pipeline function from Hugging Face Transformers

# Load the English language model from spaCy
nlp = spacy.load('en_core_web_sm')  # 'en_core_web_sm' is a small English model for various NLP tasks

# Initialize a sentiment analysis pipeline using the FinBERT model from ProsusAI
# The model is set to use a GPU ('cuda') if available; otherwise, it will use CPU by default
sentiment_analyzer = pipeline("sentiment-analysis", model="ProsusAI/finbert", device="cuda")


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


<font size="5">Function to calculate sentiment score of the aspect words in the sentence.</font>

<font size="3">The function takes in text and window size which is around the aspect words how many words to consider. Then the model pipline predicts the sentiment of the word for that context window. The idea behind context window is that whenever there is a aspect word the sentiment of that word will be around that word instead of the whole sentence.
The function returns a list which contains aspect word,it's category,sentiment label and sentiment score.</font> 


In [8]:
def absa(text, window_size=8):
    # Process the input text using the spaCy NLP model
    doc = nlp(text)
    
    # Function to find implicit aspects in a given sentence based on predefined categories and keywords
    def find_implicit_aspects(sentence, categories):
        aspects_found = []  # List to store found aspects
        sentence_text = sentence.text.lower()  # Convert the sentence text to lowercase for comparison
        for category, keywords in categories.items():  # Iterate through each category and its keywords
            for keyword in keywords:
                # Check if the keyword is present in the sentence
                if keyword.lower() in sentence_text:
                    aspects_found.append((keyword, category))  # Append the found aspect and its category
        return aspects_found  # Return the list of found aspects
    
    # Function to get a context window around a specific aspect in a sentence
    def get_windowed_context(aspect, sentence, window_size=5):
        # Tokenize the sentence using spaCy
        tokens = [token.text for token in sentence]
        aspect_tokens = nlp(aspect)  # Tokenize the aspect
        aspect_length = len(aspect_tokens)  # Get the length of the aspect in tokens
        
        # Iterate over the tokens to find the aspect
        for i in range(len(tokens) - aspect_length + 1):
            if ' '.join(tokens[i:i+aspect_length]).lower() == aspect.lower():  # Check for aspect match
                # Calculate the start and end indices for the context window
                start = max(0, i - window_size)  # Ensure start index is not negative
                end = min(len(tokens), i + aspect_length + window_size)  # Ensure end index is within bounds
                return ' '.join(tokens[start:end])  # Return the windowed context as a string
        return None  # Return None if aspect is not found
    
    aspect_sentiment = []  # List to store results of aspect sentiment analysis
    for sent in doc.sents:  # Iterate through each sentence in the document
        # Find implicit aspects in the current sentence
        implicit_aspects = find_implicit_aspects(sent, implicit_aspect_categories)
        for aspect, category in implicit_aspects:  # Iterate through found aspects
            windowed_context = get_windowed_context(aspect, sent, window_size)  # Get the context window for the aspect
            if windowed_context:  # If a context window was found
                # Analyze sentiment of the windowed context using the sentiment analysis pipeline
                sentiment = sentiment_analyzer(windowed_context)[0]  # Get the sentiment analysis result
                # Append the aspect, category, sentiment label, and sentiment score to the results list
                aspect_sentiment.append((aspect, category, sentiment['label'], sentiment['score']))
                    
    return aspect_sentiment  # Return the list of aspect sentiments


Reading the input dataset

In [9]:
df=pd.read_csv("/kaggle/input/fiqadataset/tatasentiment.csv")

Creating a column for aspect in dataframe

In [12]:
df["aspect_labelling"] = None  
df = df.astype({"aspect_labelling": 'object'})  

Running the loop for all dates news.One thing to note here is for each day there be multiple news for that company so we are considering all news individually for aspect labelling

In [14]:
for i in range(len(df)):
    sentences=list(set(df.loc[i,"news"].split(" | ")))
    aspects=[]
    for news in sentences:
        result=absa(news)
        aspects.append(result)
    df.at[i,"aspect_labelling"]=aspects

Example how it looks. If u notice there are three separate list which means there are 3 news for that day and  each list containing all sentiment label for all aspect words found in each sentence.

In [15]:
df["aspect_labelling"][0]

[[('sensex', 'stock_market', 'negative', 0.9329291582107544),
  ('Tata', 'companies', 'neutral', 0.9313399791717529),
  ('Tata Motors', 'companies', 'positive', 0.5572634935379028)],
 [('stocks', 'stock_market', 'neutral', 0.9116292595863342),
  ('Tata', 'companies', 'positive', 0.9471097588539124)],
 [('Tata', 'companies', 'neutral', 0.9239994883537292),
  ('Tata Motors', 'companies', 'neutral', 0.9239994883537292)]]

In [ ]:
df["news"][0]

In [16]:
df.head()

,Unnamed: 0,date,title,feed_type,link,news,Predicted Sentiment,Sentiment Probability,aspect_labelling
0,0,2016-10-05,Steel stocks rally as govt extends MIP on 66 s...,news-article | news-article | news-article,https://www.moneycontrol.com/news/business/sto...,"Stocks like Hindustan Zinc, Hindalco, Jindal S...",positive,0.752132,"[[(sensex, stock_market, negative, 0.932929158..."
1,1,2017-12-11,BSE Midcap index hits new high | Thyssenkrupp ...,news-article | news-article | news-article,http://www.business-standard.com/article/marke...,"Wockhdart, M&M Financial Services, Tata Commun...",positive,0.941678,"[[(Tata, companies, neutral, 0.777573883533477..."
2,2,2017-12-14,Tata Motors delivers first batch of Tigor EVs ...,news-article,https://economictimes.indiatimes.com/industry/...,"Guenter Butschek, CEO & MD, Tata Motors handed...",neutral,0.869739,"[[(Tata, companies, neutral, 0.900660037994384..."
3,3,2017-12-15,Tata Motors is driving Ace on the millennial t...,news-article,http://www.business-standard.com/article/compa...,How Tata Motors is leveraging its 12-year expe...,neutral,0.535472,"[[(Tata, companies, neutral, 0.882467091083526..."
4,4,2017-12-16,‘System error’ hits BEST’s 750 MW power tender,news-article,http://www.thehindubusinessline.com/news/syste...,Tata Power likely to benefit,neutral,0.652322,"[[(Tata, companies, neutral, 0.65232253074646)]]"


In [ ]:
df.drop(['Unnamed: 0','title','feed_type','link'],axis=1,inplace=True)

In [ ]:
df.to_csv("/kaggle/working/tata_aspect.csv",index=False)